In [ ]:
import sys
sys.path.append('../../')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import utilities

# begin by opening and examining the SP500 companies


tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
print(tickers["Symbol"].values)

In [ ]:
def retrieve_data(ticker):
    filename = "../../data/stocks/{}_full_1min_adjsplit.txt".format(ticker)

    try:
        df = pd.read_csv(filename, sep=",", header=0)
    except FileNotFoundError:
        print("{}: file not found".format(filename))
        return None

    df.columns = ["timestamp", "open", "high", "low", "close", "volume"]
    df["ticker"] = ticker
    df = utilities.timestamp_to_date_and_time(df)
    
    misses = utilities.missing_markethours(df)
    
    return misses

In [ ]:
import os

# replace tickers with the whole list
files = os.listdir('../../data/stocks')
tickers = [f.split('_')[0] for f in files if f.endswith('.txt')]

tickers.sort()

In [ ]:
all_misses = pd.DataFrame(columns=["ticker", "miss_rate"])
count = 0

for ticker in tickers:
    count += 1
    try:
        misses = retrieve_data(ticker)
        new_row = pd.DataFrame({"ticker": ticker, "miss_rate": [misses]})
        print(count, ticker, misses)
        all_misses = pd.concat([all_misses, new_row])
    except Exception as e:
        print("Error with ticker {}: {}".format(ticker, e))
        continue

In [ ]:
all_misses.to_csv("all_ticker_misses.csv", index=False)